# Hierarchical Agglomerative Clustering

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import normalize

sns.set_style('dark')

### Dataset

In [ ]:
df = pd.read_csv("data/countries-lat-long.csv",encoding='unicode_escape')

In [ ]:
df.count()

In [ ]:
df.head()

In [ ]:
#sample a subset for ease of visualization
df = df.sample(50, random_state=2023)

In [ ]:
data_scaled = normalize(df[['latitude','longitude']])
data_scaled = pd.DataFrame(data_scaled, columns=['latitude','longitude'])
data_scaled.head()

In [ ]:
plt.figure(figsize=(8, 8))

plt.scatter(df['latitude'], df['longitude'], c='r')

# Create numbered labels for each point
labels = []
for index, row in df.iterrows():
    plt.annotate(str(row['name']), xy=(row['latitude'], row['longitude']), xytext=(1, 1), textcoords='offset points')
    labels.append(row['name'])
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Countries')
plt.locator_params(nbins=10)
plt.grid()

plt.show()

We will compare linkage functions for calculating the distance between clusters below:
single, average, weighted, centroid, median, and ward.  

In [ ]:
lat_long= data_scaled.to_numpy()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

Z1 = linkage(lat_long, method='single', metric='euclidean')
Z2 = linkage(lat_long, method='complete', metric='euclidean')
Z3 = linkage(lat_long, method='average', metric='euclidean')
Z4 = linkage(lat_long, method='ward', metric='euclidean')

### Dendrogram

Notice that each distance method produces different linkages for the same data.

In [ ]:
plt.figure(figsize=(12, 20))
plt.subplot(2,2,1), dendrogram(Z1,labels=labels), plt.title('Single')
plt.subplot(2,2,2), dendrogram(Z2,labels=labels), plt.title('Complete')
plt.subplot(2,2,3), dendrogram(Z3,labels=labels), plt.title('Average')
plt.subplot(2,2,4), dendrogram(Z4,labels=labels), plt.title('Ward')
plt.show()

### Create Clusters

In [ ]:
# Creating Dendrogram for our data
# max_d = cut-off/ Threshold value
max_d = 3

plt.figure(figsize=(10, 8))
dendrogram(Z4,labels=labels), plt.title('Ward')
  
# Cutting the dendrogram at max_d
plt.axhline(y=max_d, c='k')
plt.show()

### Flat Clustering

These functions cut hierarchical clusterings into flat clusterings or find the roots of the forest formed by a cut by providing the flat cluster ids of each observation.

In [ ]:
from scipy.cluster.hierarchy import fcluster

f1 = fcluster(Z4, 3, criterion='maxclust')

print(f"Clusters: {f1}")

### Sklearn Heirarchical Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

Z5 = AgglomerativeClustering(n_clusters=3, linkage='ward')

Z5.fit_predict(lat_long)

print(Z5.labels_)

In [ ]:
df['labels'] = Z5.labels_

In [ ]:
df.head()

### Plot Clusters

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

scatter = ax.scatter(df['latitude'], df['longitude'], c=Z5.labels_, cmap='rainbow')

legend = ax.legend(*scatter.legend_elements(), title="Clusters", bbox_to_anchor=(1, 1))
ax.add_artist(legend)

for index, row in df.iterrows():
    plt.annotate(str(row['name']), xy=(row['latitude'], row['longitude']), xytext=(1, 1), textcoords='offset points')
    labels.append(row['name'])

plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Countries')
plt.locator_params(nbins=10)
plt.grid()

plt.show()